In [1]:
%autoreload 2

In [2]:
from collections import defaultdict
import difflib
import gzip
import os
import pickle
import sys
import time
import typing

from IPython.display import display, Markdown, HTML
import numpy as np
import tqdm.notebook as tqdm

sys.path.append(os.path.abspath('..'))
sys.path.append(os.path.abspath('../src'))
from src.ast_utils import load_games_from_file, _extract_game_id

In [3]:
import openai
openai.api_key = os.getenv('OPENAI_API_KEY')

In [13]:
game_texts = list(load_games_from_file('../dsl/interactive-beta.pddl'))
game_texts_with_comments = list(load_games_from_file('../dsl/interactive-beta-with-comments.pddl'))

In [ ]:
html_diff = difflib.HtmlDiff(wrapcolumn=60)

def diff_by_index(index: int):
    diff = html_diff.make_file(game_texts[index].splitlines(), game_texts_with_comments[index].splitlines())  #, context=True, numlines=0)
    display(HTML(diff))


for i in range(len(game_texts)):
    if game_texts[i] != game_texts_with_comments[i]:
        print(i)
        print(game_texts[i][:42])
        diff_by_index(i)

In [15]:
SETUP_COMMENNT = '; SETUP:'
PREFERENCE_COMMENT = '; PREFERENCE:'
TERMINAL_COMMENT = '; TERMINAL:'
SCORING_COMMENT = '; SCORING:'
COMMENT_PREFIXES = [SETUP_COMMENNT, PREFERENCE_COMMENT, TERMINAL_COMMENT, SCORING_COMMENT]


game_texts_with_comments = list(load_games_from_file('../dsl/interactive-beta-with-comments.pddl',
    remove_comments=False, comment_prefixes_to_keep=COMMENT_PREFIXES))


In [16]:
comments_by_type = defaultdict(list)
for game_text in game_texts_with_comments:
    for comment_prefix in COMMENT_PREFIXES:
        comment_start_index = game_text.find(comment_prefix)
        while comment_start_index != -1:
            commend_end_index = game_text.find('\n', comment_start_index)
            comment = game_text[comment_start_index:commend_end_index]
            comment_type = comment_prefix[2:-1]
            comments_by_type[comment_type].append(comment)
            comment_start_index = game_text.find(comment_prefix, commend_end_index)


In [61]:
GAME_START = '(define'
SETUP_SECTION = '(:setup'
PREFERENCES_SECTION = '(:constraints'
TERMINAL_SECTION = '(:terminal'
SCORING_SECTION = '(:scoring'

DEFAULT_CODEX_MODEL = "code-davinci-002"
DEFAULT_TEMPERATURE = 0.67
DEFAULT_MAX_TOKENS = 512
DEFAULT_STOP_SEQUENCES = [';', PREFERENCES_SECTION, TERMINAL_SECTION, SCORING_SECTION]
MAX_N = 10


DEFAULT_COMPLETION_KWARGS = dict(
    model=DEFAULT_CODEX_MODEL,
    temperature=DEFAULT_TEMPERATURE,
    max_tokens=DEFAULT_MAX_TOKENS,
    stop=DEFAULT_STOP_SEQUENCES,
)


def generate_codex_completions(prompt: str, suffix: str, n: int, 
    completion_kwargs: typing.Optional[typing.Dict[str, typing.Any]],
    ):

    if n > MAX_N:
        raise ValueError(f'n must be <= {MAX_N}')

    if completion_kwargs is None:
        completion_kwargs = DEFAULT_COMPLETION_KWARGS

    else:
        kwargs = DEFAULT_COMPLETION_KWARGS.copy()
        kwargs.update(completion_kwargs)
        completion_kwargs = kwargs

    completion_kwargs['prompt'] = prompt
    completion_kwargs['suffix'] = suffix
    completion_kwargs['n'] = n

    return openai.Completion.create(**completion_kwargs)

In [99]:
GAMES_WITH_SETUP = set([i for i, g in enumerate(game_texts_with_comments) if SETUP_SECTION in g])
GAMES_WITH_SETUP_LIST = list(GAMES_WITH_SETUP)
GAMES_WITH_TERMINAL = set([i for i, g in enumerate(game_texts_with_comments) if TERMINAL_SECTION in g])
GAMES_WITH_TERMINAL_LIST = list(GAMES_WITH_TERMINAL)


def _verify_valid_comment(comment: str, section_comment_key: str) -> str:
    if not comment.startswith(section_comment_key):
        comment = f'{section_comment_key} {comment}'
    return comment


def generate_setup_prompt_and_suffix(comment: str, include_terminal: bool = False):
    prompt = f'(define (game {np.random.randint(16 ** 12):012x}{np.random.randint(16 ** 12):012x}-{np.random.randint(120, 300)}) (:domain {np.random.choice(["few", "medium", "many"])}-objects-room-v1)\n'
    comment = _verify_valid_comment(comment, SETUP_COMMENNT)
    prompt += comment + '\n'
    prompt += SETUP_SECTION + ' '

    suffix = \
f''')
{PREFERENCES_SECTION} (and <preferences> ))
{TERMINAL_SECTION + " <terminal> )" if include_terminal else ''}
{SCORING_SECTION} <scoring> )
)'''

    return prompt, suffix


def generate_preference_prompts_and_suffixes(comment_or_comments: typing.Union[str, typing.List[str]], include_terminal: bool = False):
    if isinstance(comment_or_comments, str):
        comment_or_comments = [comment_or_comments]

    prompts = []
    suffixes = []

    for i, comment in enumerate(comment_or_comments):
        comment = _verify_valid_comment(comment, PREFERENCE_COMMENT)

        if i == 0:
            prompt = f'{PREFERENCES_SECTION} (and\n\t{comment}\n\t'
        else:
            prompt = f'\t{comment}\n\t'        

        prompts.append(prompt)

    for i in range(len(comment_or_comments)):
        if i == 0:
            suffix = f'''))
{TERMINAL_SECTION + " <terminal> )" if include_terminal else ''}
{SCORING_SECTION} <scoring> )
)'''
        else:
            relevant_prompt = prompts[-i]
            suffix = f'{relevant_prompt}<preference>\n{suffixes[i - 1]}'
        
        suffixes.append(suffix)

    suffixes.reverse()
    return prompts, suffixes


def fix_parentheses(text: str) -> str:
    while text.count('(') > text.count(')'):
        text += ')'

    while text.count(')') > text.count('('):
        last_close_paren = text.rfind(')')
        text = text[:last_close_paren] + text[last_close_paren+1:]

    return text


In [67]:
setup_game_indices = np.random.choice(GAMES_WITH_SETUP_LIST, 3)
setup_prompt, setup_suffix = generate_setup_prompt_and_suffix('place the hexagonal bin on a chair in the middle of the room')

full_prompt = '\n'.join([game_texts_with_comments[i] for i in setup_game_indices] + [setup_prompt])

In [68]:
setup_result = generate_codex_completions(full_prompt, setup_suffix, 1, {'temperature': 0.95})
setup_text = fix_parentheses(setup_result['choices'][0]['text'])
game_in_progress = setup_prompt + setup_text
print(game_in_progress)

In [73]:
setup_text = fix_parentheses(setup_result['choices'][0]['text'])
game_in_progress = setup_prompt + setup_text
print(game_in_progress)

(define (game 8936caf9bc001d51a78ae1b9-252) (:domain many-objects-room-v1)
; SETUP: place the hexagonal bin on a chair in the middle of the room
(:setup (and 
    (exists (?c - chair ?h - hexagonal_bin) (and 
        (game-conserved (and 
            (on ?c ?h)
            (adjacent_side ?c front room_center)
        ))
    ))
)



In [103]:
pref_prompts, pref_suffixes = generate_preference_prompts_and_suffixes(['count throws of a basketball that hit the bin', "count throws of a basketball that hit the chair"])
preference_game_indices = np.random.choice(np.arange(len(game_texts_with_comments)), 3)
full_pref_0_prompt = '\n'.join([game_texts_with_comments[i] for i in preference_game_indices] + [game_in_progress, pref_prompts[0]])
pref_0_suffix = pref_suffixes[1]
pref_0_result = generate_codex_completions(full_pref_0_prompt, pref_0_suffix, 1, {'temperature': 0.8})
pref_0_text = fix_parentheses(pref_0_result['choices'][0]['text'])
print(pref_0_text)

(preference ballThrownThroughBin:basketball (exists (?a - basketball ?c - chair ?h - hexagonal_bin)
	(then 
		(once (agent_holds ?a))
		(hold (and (in_motion ?a) (not (agent_holds ?a))))
		(once (and
			(not (in_motion ?a))
			(touch ?a ?h)
			)
		))
	))
	
	
